In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader, TextLoader
import magic
import os
import nltk

In [2]:
# let's load the texts that we want to analyze
loader = DirectoryLoader(path="../data/texts/", glob="**/*.txt")
docs = loader.load()
len(docs)

2

In [3]:
docs

[Document(page_content='\ufeffThe Project Gutenberg eBook of The Art of War, by Sun Tzŭ\n\nThis eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.\n\nTitle: The Art of War\n\nAuthor: Sun Tzŭ\n\nTranslator: Lionel Giles\n\nRelease Date: May 1994 [eBook #132] [Most recently updated: October 16, 2021]\n\nLanguage: English\n\n** START OF THE PROJECT GUTENBERG EBOOK THE ART OF WAR ***\n\nSun Tzŭ\n\non\n\nThe Art of War\n\nTHE OLDEST MILITARY TREATISE IN THE WORLD Translated from the Chinese with Introduction and Critical Notes\n\nBY\n\nLIONEL GILES, M.A.\n\nAssistant in the Department of Oriental Printed Books 

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents=docs)
len(text_chunks)

## create text embeddings

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key = os.environ['OPENAI_API_KEY'])

In [24]:
docsearch = Chroma.from_documents(text_chunks, embeddings, persist_directory="../db")

qa = RetrievalQA.from_chain_type(llm = OpenAI(temperature=0.9),
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=True
                                 )

Using embedded DuckDB with persistence: data will be stored in: ../db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [8]:
query = "How to win a war?"
result = qa({
    "query": query
})
print(f"Answer: {result['result']}")

Answer:  He will win who has military capacity and is not interfered with by the sovereign, knows the enemy and knows himself, brings war material with him from home, but forages on the enemy, and maintains an army without impoverishing the people.


In [9]:
result

{'query': 'How to win a war?',
 'result': ' He will win who has military capacity and is not interfered with by the sovereign, knows the enemy and knows himself, brings war material with him from home, but forages on the enemy, and maintains an army without impoverishing the people.',
 'source_documents': [Document(page_content='(5) He will win who has military capacity and is not interfered with by the sovereign.\n\n[Tu Yu quotes Wang Tzŭ as saying: "It is the sovereign’s function to give broad instructions, but to decide on battle it is the function of the general." It is needless to dilate on the military disasters which have been caused by undue interference with operations in the field on the part of the home government. Napoleon undoubtedly owed much of his extraordinary success to the fact that he was not hampered by central authority.]\n\nVictory lies in the knowledge of these five points.\n\n[Literally, “These five things are knowledge of the principle of victory.”]\n\n18. Hen

In [13]:
for d in result['source_documents']:
    try:
        index = d.page_content.index("He will win who has")
    except Exception as e:
        index = -1

    if index >=0:
        print(f"{d.page_content[index:index+20]}")

He will win who has 
He will win who has 
He will win who has 


In [14]:
from pprint import pprint
pprint(result['result'])

(' He will win who has military capacity and is not interfered with by the '
 'sovereign, knows the enemy and knows himself, brings war material with him '
 'from home, but forages on the enemy, and maintains an army without '
 'impoverishing the people.')


In [18]:
def governance_question(query):
    if len(query.strip()) > 0:
        result = qa({
            "query": query
        })
        pprint(f"Answer: {result['result']}")

In [25]:
governance_question("How to win a war?")

('Answer:  He will win who has military capacity and is not interfered with by '
 'the sovereign. It is important to know both the enemy and oneself in order '
 'to win a war.')


In [26]:
governance_question("How to best govern a group of people?")

('Answer:  A wise prince should choose wise men in his state and give them the '
 'liberty to speak the truth to him. He should then question them and listen '
 'to their opinions before forming his own conclusions. He should carry '
 'himself in such a way that each of the councillors know that the more freely '
 'they shall speak, the more they shall be preferred. Outside of this group, '
 'he should listen to no one and pursue the thing resolved on, being steadfast '
 'in his resolutions. Otherwise, he risks being overthrown by flatterers or '
 'changing his opinion too much and falling into contempt.')


In [27]:
governance_question("How to lose a war?")

('Answer:  Repeat the same tactics that gained you one victory and try to '
 'fight what is strong, rather than what is weak.')


In [28]:
governance_question("What are the best weapons to use in a war?")

('Answer:  The author suggests that military weapons are the best means to '
 'punish violence and cruelty, to give peace to troublous times, to remove '
 'difficulties and dangers, and to succour those who are in peril. The author '
 'also states that military chastisement should not be allowed to fall into '
 'abeyance in the Empire.')


In [29]:
governance_question("What is a war?")

('Answer:  War may be defined as punishment, which is one of the functions of '
 'government. It involves the wielding of huge armies, the throwing down of '
 'fortified cities, the hauling of women and children into captivity, and the '
 'beheading of traitors, all of which is done by officials. Its aim is to get '
 'rid of wicked people and to give comfort and relief to the good.')
